# Openpose 1.7.0 Demo

In [ ]:
# 最初に、ランタイムの種類を「GPU」に変更してください。（ランタイム＞ランタイムのタイプを変更）
# GPUが有効になっていることを確認します。

# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

In [ ]:
# Openpose バージョン指定タグ
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

In [ ]:
# CMakeが古いとOpenpose（CUDA10)が失敗するので、バージョンを確認します。（BugfixはCMake 3.12.3）
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
! cmake --version

In [ ]:
# 2021.03.03時点の最新CMakeを再ビルド（15分くらい）
# Rebuild the latest CMake as of 2021.03.03 (about 15 minutes)
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

In [ ]:
# ライブラリインストール
# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [ ]:
# Openposeのコードをclone
# Clone Openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

In [ ]:
# build用ディレクトリを作成
# Create build directory
! cd openpose && mkdir build && cd build

In [ ]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# 上記インストール手順のシナリオ１でインストール実行
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake .. 

# COCOモデルのDLオプション付き(「# ! cd」の部分を「! cd」に変更してください。)
# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# MPIモデルのDLオプション付き（同上）
# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

In [ ]:
# Openposeのビルド（15分くらい）
# Openpose Building
! cd openpose/build && make -j`nproc`
# outputフォルダ作成
! cd openpose && mkdir output

In [ ]:
# デモ動画を解析
# 出力された解析結果は、Colab画面の左メニューのフォルダアイコンから「openpose/output」以下に配置されます。
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --display 0  --write_video ./output/openpose.avi

In [ ]:
# デモ動画の結果をffmpegで再エンコードして表示します。少し時間がかかります。
# The result of the demo video is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "./openpose/output/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

In [ ]:
from google.colab import drive
import os

# Googleドライブマウント
# マウントすると、アクセス許可が求められるので、許可して、キーをコピーして、Colab側の入力欄にCtrl+Vで貼り付けて、Enter。
# mount Google Drive
# When you mount it, you will be asked for permission, so allow it, copy the key, paste(Ctrl+V) it in the input field on the Colab side, and press Enter.
drive.mount('/gdrive')

# Google Drive のルートディレクトリ
# Google Drive root directory
root_path = "/gdrive/My Drive/"

# Googleドライブ上で解析したい動画パス（この値を好きな値に変更する事で、ご自分のGoogleドライブ上の動画を解析できます）
# Video path that you want to analyze on Google Drive (You can analyze the video on your own Google Drive by changing this value to a value you like)
file_path = "autotrace/input.mp4"

# Googleドライブ上の動画を解析（Googleドライブのルートディレクトリに解析結果の「openpose.avi」が出力されます）
# Analyze video on Google Drive (The analysis result "openpose.avi" is output to the root directory of Google Drive)
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi"

In [ ]:
# Googleドライブ上の解析結果動画をffmpegで再エンコードして表示します。少し時間がかかります。
# The analysis result video on Google Drive is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "$root_path/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

In [ ]:
# 指定した人数分のみ解析したい場合、「number_people_max」の後ろに人数を指定して下さい
# If you want to analyze only the specified number of people, please specify the number of people after "number_people_max"
# 結果はひとつ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code one level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --number_people_max 1

In [ ]:
# 手の解析も一緒に行いたい場合、「--hand」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は２つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code two level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --hand

In [ ]:
# 顔の解析も一緒に行いたい場合、「--face」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は３つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code three level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --face